In [1]:
import os
import re
import json
import tempfile
import folium
import ipysheet
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import branca.colormap as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets

from glob import glob
from pathlib import Path
from folium import plugins
from pyproj import Transformer
from ipyfilechooser import FileChooser
from ipysheet import from_dataframe
from rasterio.warp import calculate_default_transform, reproject, Resampling
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl, AwesomeIcon, GeoData, basemap_to_tiles
from ipyleaflet import LegendControl, FullScreenControl, MarkerCluster

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import TileClient, get_leaflet_tile_layer
from localtileserver import examples, helpers
from ipyleaflet import Map, SplitMapControl
import rasterio as rio

In [2]:
#https://girder.github.io/large_image/tilesource_options.html#style

# path = "/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_3"
# filename = "WV02_20130813_M1BS_1030010026D4A600-sr-02m.tif"

ccdc_cir_style_list = [{'band': 4, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
ccdc_rgb_style_list = [{'band': 3, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 1, 'palette': '#00f', 'min': 0, 'max': 2500}]

chm_palette = [
    '#636363',
    '#fc8d59',
    '#fee08b','#fee08b',
    '#ffffbf','#ffffbf',
    '#d9ef8b','#d9ef8b',
    '#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60',
    '#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850',
    '#005a32'
]

# cloud_palette = ['#c2523c','#005a32']
chm10_style_list = [{'min': 0, 'max': 10.25, 'scheme': 'discrete', 'clamp': True, 'palette': chm_palette}]
chm10_style_list_ft = [{'min': 0, 'max': 33.63, 'scheme': 'discrete', 'clamp': True, 'palette': chm_palette}]
cm_style_list = [{'palette': ['#00000000','#cccccc']}]
# chmpath = '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20100804_M1BS_103001000617C000-sr-02m/WV02_20100804_M1BS_103001000617C000-sr-02m.cnn-chm-v1.tif'

# lvis_RH098_path = '/explore/nobackup/people/mmacande/LVIS_annual/LVISF2_ABoVE2017_2019_RH098_mean_30m.tif'
# lvis_RH098_client = TileClient(lvis_RH098_path)

In [3]:
esri = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri.base = True
carto = basemap_to_tiles(basemaps.CartoDB.Positron)
carto.base = True

In [4]:
def build_leaflet_layer(name, path, clamp, scheme, dtype, style):
    client = TileClient(path)
    layer = get_leaflet_tile_layer(
        client, name=name, clamp=clamp,
        scheme=scheme, dtype=dtype, style=style)
    return layer

In [5]:
from ipywidgets import Layout
defaultLayout=Layout(width='90%', height='800px')


In [6]:
inf_2011_26905_ndsm = build_leaflet_layer('2011 Infrastructure utm5', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/infrastructure_2011/infrastructure_2011_26905_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
inf_2011_26906_ndsm = build_leaflet_layer('2011 Infrastructure utm6', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/infrastructure_2011/infrastructure_2011_26906_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
inf_2011_26907_ndsm = build_leaflet_layer('2011 Infrastructure utm7', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/infrastructure_2011/infrastructure_2011_26907_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
yukon_flats_2009_chm = build_leaflet_layer('2009 Yukon Flats', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/yukon_flats_2009/yukon_flats_2009_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
kenai_2008_chm = build_leaflet_layer('2008 Kenai Peninsula', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/Kenai_2008/Kenai_2008_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
unalakleet_2019_chm = build_leaflet_layer('2019 Unalakleet', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/Unalakleet_2019/Unalakleet_2019_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
NSB_lidar_Anaktuvuk_Pass_2019_chm = build_leaflet_layer('2019 Anaktuvuk Pass', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/NSB_lidar_Anaktuvuk_Pass_2019/NSB_lidar_Anaktuvuk_Pass_2019_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
fairbanks_ql1_2017_chm = build_leaflet_layer('fairbanks_ql1_2017_chm', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/fairbanks_ql1_2017/fairbanks_ql1_2017_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
fairbanks_ql2_2017_chm = build_leaflet_layer('fairbanks_ql2_2017_chm', '/explore/nobackup/projects/dem/AK_IFSAR/dggs.alaska.gov/public_lidar/fairbanks_ql2_2017/fairbanks_ql2_2017_chm.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
lvis_2017_2019_rh098 = build_leaflet_layer('lvis_2017_2019_rh098', '/explore/nobackup/people/mmacande/LVIS_annual/LVISF2_ABoVE2017_2019_RH098_mean_30m.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
alaska_ifsar_chm = build_leaflet_layer('Alaska IFSAR CHM', '/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_chm_m_20221222_cog_west.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})
gliht_2018_32606_chm = build_leaflet_layer('2018 GLiHT 32606', '/explore/nobackup/people/pmontesa/userfs02/data/gliht/chm/gliht_chm_2018_32606.tif',
                                          True, 'linear', 'uint16', {'bands': chm10_style_list})


In [7]:
def draw_chm_ref():    
    m = Map(center=(65,-160), zoom=4, layers=[carto, esri], scroll_wheel_zoom=True)
    control = LayersControl(position='topright')
    m.add_layer(alaska_ifsar_chm)
    # m.add_layer(lvis_2017_2019_rh098)
    # m.add_layer(inf_2011_26905_ndsm)
    # m.add_layer(inf_2011_26906_ndsm)
    # m.add_layer(inf_2011_26907_ndsm)
    # m.add_layer(yukon_flats_2009_chm)
    # m.add_layer(kenai_2008_chm)
    # m.add_layer(unalakleet_2019_chm)
    # m.add_layer(NSB_lidar_Anaktuvuk_Pass_2019_chm)
    # m.add_layer(fairbanks_ql1_2017_chm)
    # m.add_layer(fairbanks_ql2_2017_chm)
    m.add_layer(gliht_2018_32606_chm)
    
    m.add_control(FullScreenControl())
    m.add_control(ScaleControl(imperial=False))
    m.add_control(LegendControl({
        '<0.25m': '#636363',
        '<0.50m': '#fc8d59',
        '<1.0 m': '#fee08b',
        '<1.5 m': '#ffffbf',
        '<2.0 m': '#d9ef8b',
        '<5.0 m': '#91cf60',
        '<10 m': '#1a9850',
        '>10 m': '#005a32'},
        position='bottomleft'))
    m.add_control(control)

    m
    return m

        

In [8]:
map = draw_chm_ref()
# map = chmExample.apply(draw_chm_cir, axis=1).values[0]
map.layout.width = '90%'
map.layout.height = '800px'

map

Map(center=[65, -160], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…